In [1]:
import torch
import numpy as np
import sys
import os


from dataloader import MyCustomDataset
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim


from model_nn import  Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [2]:
import numpy as np

# Load the NumPy arrays
disease_SB = np.load('/home/karansingh/Documents/summer-term/KARAN_ECG/data_prep/disease_SB.npy')
disease_SR = np.load('/home/karansingh/Documents/summer-term/KARAN_ECG/data_prep/disease_SR.npy')
disease_ST = np.load('/home/karansingh/Documents/summer-term/KARAN_ECG/data_prep/disease_ST.npy')



In [3]:
disease_SB=disease_SB.reshape(5000,60000)
disease_SR=disease_SR.reshape(5000,60000)
disease_ST=disease_ST.reshape(5000,60000)


In [4]:
# convert nan to zero
disease_SB = np.nan_to_num(disease_SB)
disease_SR = np.nan_to_num(disease_SR)
disease_ST = np.nan_to_num(disease_ST)


In [5]:
images = np.concatenate((disease_SB, disease_SR, disease_ST))
labels = np.concatenate((np.zeros(disease_SB.shape[0]), np.ones(disease_SR.shape[0]), np.ones(disease_ST.shape[0])*2))
labels.shape

(15000,)

In [6]:
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.2, random_state=42)
train_dataset = MyCustomDataset(images_train, labels_train)
test_dataset = MyCustomDataset(images_test, labels_test)
# Define a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
class Model(nn.Module):
    """Base model with 5 layers"""
    def __init__(self, in_features=60000, h1=2048, h2=512,
                 h3=264, h4=68, h5=16,
                 out_features=3):
        """Initialize an instance of the model"""
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, h4)
        self.fc5 = nn.Linear(h4, h5)
        self.out = nn.Linear(h5, out_features)

    def forward(self, x):
        """Forward method for the layers in the model"""
        # Using softmax in the output layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.out(x)
        return x


In [8]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Print the memory usage
def print_memory_usage():
    allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
    reserved = torch.cuda.memory_reserved(device) / (1024 ** 2)
    print(f"Memory Allocated: {allocated:.2f} MiB")
    print(f"Memory Reserved: {reserved:.2f} MiB")

# Example usage
print_memory_usage()

Memory Allocated: 0.00 MiB
Memory Reserved: 0.00 MiB


In [9]:
# emputy the cache
torch.cuda.empty_cache()

In [10]:
#orignal
import torch.nn.functional as F
torch.manual_seed(42)  # Set seed for reproducibility
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model__ = Model().to(device)

criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model__.parameters(), lr=0.001)  # Adam optimizer with learning rate of 0.001  
def accuracy(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

num_epochs = 25
test_losses_list = []
train_losses_list = []
accuracy_train_list = []
accuracy_test_list = []

for epoch in range(num_epochs):
    model__.train()
    running_loss, accuracy_train = 0.0, 0.0
    for i, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model__(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        accuracy_train += accuracy(labels, outputs.argmax(dim=1))
    train_losses_list.append(running_loss / len(train_dataloader))
    
    # Just calculating the test loss and accuracy
    with torch.no_grad():
        model__.eval()
        test_loss, accuracy_test = 0.0, 0.0
        for i, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model__(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            accuracy_test += accuracy(labels, outputs.argmax(dim=1))
    test_losses_list.append(test_loss / len(test_dataloader))
    
    accuracy_train_list.append(accuracy_train / len(train_dataloader))
    accuracy_test_list.append(accuracy_test / len(test_dataloader))

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss / len(train_dataloader):.4f}, Test Loss: {test_loss / len(test_dataloader):.4f}, Train Accuracy: {accuracy_train / len(train_dataloader):.4f}, Test Accuracy: {accuracy_test / len(test_dataloader):.4f}')

Epoch [1/25], Train Loss: 0.7018, Test Loss: 0.5102, Train Accuracy: 70.0917, Test Accuracy: 81.8041
Epoch [2/25], Train Loss: 0.3445, Test Loss: 0.5115, Train Accuracy: 87.8083, Test Accuracy: 82.4357
Epoch [3/25], Train Loss: 0.1874, Test Loss: 0.5765, Train Accuracy: 93.9333, Test Accuracy: 82.7017
Epoch [4/25], Train Loss: 0.1005, Test Loss: 0.7880, Train Accuracy: 96.6750, Test Accuracy: 82.2252
Epoch [5/25], Train Loss: 0.1128, Test Loss: 0.7401, Train Accuracy: 96.6667, Test Accuracy: 82.0479
Epoch [6/25], Train Loss: 0.0814, Test Loss: 1.0995, Train Accuracy: 97.7750, Test Accuracy: 83.0230
Epoch [7/25], Train Loss: 0.0592, Test Loss: 1.0086, Train Accuracy: 98.3917, Test Accuracy: 82.3692
Epoch [8/25], Train Loss: 0.0687, Test Loss: 0.9838, Train Accuracy: 97.9667, Test Accuracy: 82.0368
Epoch [9/25], Train Loss: 0.0282, Test Loss: 1.1739, Train Accuracy: 99.1667, Test Accuracy: 81.7154
Epoch [10/25], Train Loss: 0.0579, Test Loss: 0.9897, Train Accuracy: 98.7500, Test Accurac